# HOMEWORK ASSIGNMENT 5

### QUESTION 1

In [56]:
import pandas as pd 
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F 

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/20 15:15:20 WARN Utils: Your hostname, pepper resolves to a loopback address: 127.0.1.1; using 192.168.178.64 instead (on interface wlp2s0)
24/02/20 15:15:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 15:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version


'3.5.0'

### QUESTION 3 

In [26]:
# read in a subset of the data
url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz'
df_csv = pd.read_csv(url, nrows=10000)

In [27]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dispatching_base_num    10000 non-null  object 
 1   pickup_datetime         10000 non-null  object 
 2   dropOff_datetime        10000 non-null  object 
 3   PUlocationID            9953 non-null   float64
 4   DOlocationID            9953 non-null   float64
 5   SR_Flag                 0 non-null      float64
 6   Affiliated_base_number  9966 non-null   object 
dtypes: float64(3), object(4)
memory usage: 547.0+ KB


In [28]:
df_csv.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [29]:
spark_df=spark.createDataFrame(df_csv)

In [33]:
spark_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [35]:
spark_df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

### CONVERT SCHEMA 

In [39]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.DoubleType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [46]:
# I couldn't read it directly from the URL ... maybe because it was a .gz file
spark_df_full = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [47]:
# REPARTITION into 6 partitions
spark_df_full = spark_df_full.repartition(6)


In [48]:
spark_df_full.write.parquet('fhvhv/2019/10/', mode="overwrite")

In [49]:
!ls -lh fhvhv/2019/10/

total 36M
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00000-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00001-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00002-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00003-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00004-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu 5.9M Feb 20 16:01 part-00005-4db25f05-9957-47c1-9701-8969f26a17a6-c000.snappy.parquet
-rw-r--r-- 1 lulu lulu    0 Feb 20 16:01 _SUCCESS


### QUESTION 3 

In [53]:
# How many taxi trips were there on the 15th of October?
spark.sql('''
SELECT count(*)
FROM parquet.`fhvhv/2019/10/`
WHERE date(pickup_datetime) = '2019-10-15'
''').show()


+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [61]:
# FIRST REGISTER THE TABLE
spark_df_full.createOrReplaceTempView('fhvhv')


In [54]:
# query the table 
spark.sql('''
SELECT count(*)
FROM fhvhv
WHERE date(pickup_datetime) = '2019-10-15'
''').show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



### QUESTION 4 
Since there is no time diff function, we cast the timestamp column to a long value which gets time in seconds, and then divides it by a value to get appropriate units.

In [57]:
spark_df_full = spark_df_full.withColumn('duration_hr', \
                                         (F.col('dropOff_datetime').cast('long') - \
                                          F.col('pickup_datetime').cast('long')) / 3600)


In [58]:
spark_df_full.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|         duration_hr|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B02243|2019-10-01 12:38:00|2019-10-01 14:02:00|         202|          11|   NULL|                LX-176|                 1.4|
|              B03062|2019-10-01 15:43:10|2019-10-01 15:46:20|         264|          32|   NULL|                B03062| 0.05277777777777778|
|              B01984|2019-10-02 11:11:00|2019-10-02 11:27:00|         264|         221|   NULL|                B01984| 0.26666666666666666|
|              B01051|2019-10-01 11:52:26|2019-10-01 12:04:22|         264|         168|   NULL|                B01051|  0.1988888888888889|
|            

In [ ]:
# RE-REGISTER THE TABLE --> Otherwise new columns are not available
spark_df_full.createOrReplaceTempView('fhvhv')

In [62]:
spark.sql('''
SELECT MAX(duration_hr)
FROM fhvhv
''').show()

+----------------+
|max(duration_hr)|
+----------------+
|        631152.5|
+----------------+



## QUESTION 6 
Broadcasting 

In [ ]:
lookup_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'

In [64]:
df_zones = spark.read.parquet('../5-batch/zones/', header=True)

In [65]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [71]:
df_result = spark_df_full.join(df_zones, df_zones.LocationID == spark_df_full.PUlocationID)

In [73]:
df_result.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+---------+----------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|         duration_hr|LocationID|  Borough|            Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+---------+----------------+------------+
|              B02243|2019-10-01 12:38:00|2019-10-01 14:02:00|         202|          11|   NULL|                LX-176|                 1.4|       202|Manhattan|Roosevelt Island|   Boro Zone|
|              B03062|2019-10-01 15:43:10|2019-10-01 15:46:20|         264|          32|   NULL|                B03062| 0.05277777777777778|       264|  Unknown|              NV|         N/A|
|              B01984|2019-10-02 11:11:0

In [74]:
# Register the new table 
df_result.createOrReplaceTempView('fhvhv_zones')

In [81]:
spark.sql('''
SELECT zone, count(*)
FROM fhvhv_zones
GROUP BY zone
ORDER BY 2 ASC
''').show()

+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

